In [1]:
# Import Library
import pandas as pd
import os
import re
import pymongo
from pymongo import MongoClient
from dotenv import load_dotenv
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain.prompts import PromptTemplate
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

In [2]:
# Load Environment Variables
load_dotenv(override=True)
MONGODB_URI = os.getenv("MONGODB_URI")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY") 

In [3]:
# Initialize Google Embeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=GOOGLE_API_KEY
)

In [4]:
# Model Configuration dengan Gemini Pro
llm_gemini = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    google_api_key=GOOGLE_API_KEY,
    temperature=0,
    max_tokens=1000,
    convert_system_message_to_human=True # Diperlukan agar prompt sistem bekerja optimal
)

In [5]:
# MongoDB Connection
client = MongoClient(MONGODB_URI)
collection = client['chatbotproject_db']['faq_tb']
index_name = 'vector_index'

In [6]:
# Vector Store Configuration
vector_store = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embeddings,
    index_name='vector_index'
)

In [7]:
# Chatbot Prompt Template
BPJS_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Anda adalah Asisten Digital Resmi BPJS Kesehatan dan BPJS Ketenagakerjaan.
    Tugas Anda sebagai Customer Service adalah menjawab pertanyaan peserta berdasarkan data FAQ resmi yang telah disediakan.

    Tanggung jawab utama:
    - Memberikan jawaban yang akurat dan informatif sesuai dengan kategori FAQ BPJS Kesehatan dan BPJS Ketenagakerjaan.
    - Menyampaikan informasi dengan bahasa yang profesional, jelas, dan mudah dipahami oleh seluruh lapisan masyarakat.
    - Memberikan panduan teknis atau prosedur (jika diperlukan) dalam format langkah-langkah yang berurutan atau poin-poin.
    - Menanggapi pertanyaan atau peserta dengan solusi yang relevan dari konteks FAQ yang tersedia.

    Aturan jawaban:
    1. Jawab hanya berdasarkan informasi yang tersedia dalam konteks FAQ resmi yang diberikan.
    2. Jika pertanyaan tidak relevan atau tidak tercakup dalam FAQ, jawab dengan sopan: "Mohon maaf, saya tidak memiliki informasi mengenai hal tersebut. Saya hanya dapat menjawab pertanyaan seputar layanan BPJS Kesehatan dan BPJS Ketenagakerjaan."
    3. Jangan mengarang jawaban atau memberikan informasi di luar dokumen sumber.
    4. Jangan mencantumkan link eksternal atau informasi promosi di luar konteks.
    5. Gunakan format poin atau numerik bila menjelaskan prosedur atau langkah-langkah.

    Konteks resmi (dari FAQ):
    {context}

    Pertanyaan Peserta: {question}

    Jawaban Profesional:
    """
)

In [8]:
# RerievalQA Configuration
qa = RetrievalQA.from_chain_type(
    llm=llm_gemini,
    chain_type="stuff",
    retriever=vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5, "score_treshold":0.85 
                       }  # Mengambil 5 dokumen teratas

    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": BPJS_PROMPT}
)

# Cleaning Function
def clean_answer(raw_answer):
    """Membersihkan jawaban dari referensi dan format khusus"""
    # Format daftar bernomor
    formatted = re.sub(r'(\d+\.)\s', r'\n\1 ', raw_answer)
    # Hapus karakter khusus
    cleaned = re.sub(r'[*_]{2}', '', formatted)
    return cleaned.strip()

# Answer Function
def ask(query):
    try:
        # Proses query langsung tanpa validasi awal
        result = qa.invoke({"query": query})
        if not result['source_documents']:
            return "Informasi tidak ditemukan dalam database resmi."
        # Pembersihan jawaban akhir
        return clean_answer(result['result'])

    except Exception as e:
        return f"Terjadi kesalahan sistem: {str(e)}\nSilakan coba lagi."

# Answer from Document Function
faq_df = pd.read_csv('final_dataset_clean.csv')
def answer_from_doc(question):
    row = faq_df[faq_df['question'].str.lower() == question.lower()]
    if not row.empty:
        return row.iloc[-1]['answer']
    else:
        return "Tidak ada jawaban."

In [9]:
# TEST CASE 1: Pertanyaan di dalam konteks (formal dan informal)
print("\n\n UJI COBA 1: Pertanyaan yang Seharusnya Bisa Dijawab karena berasal dari dataset")
questions = [
    "Bagaimana cara menginstall aplikasi Jamsostek Mobile (JMO)?",
    "Bagaimana cara mengajukan klaim JHT melalui JMO jika memiliki lebih dari 1 kartu?",
    "Promo apa saja yang tersedia pada aplikasi Jamsostek Mobile (JMO)?",
    "Jika anggota baru di JMO, apakah harus mendaftar di MotionPay?",
    "Apakah itu Promo?",
    "Apakah itu MotionPay?", 
    "Bagaimana jika lupa PIN EPS?",
    "Apakah sandi/PIN harus diubah secara berkala di kedua aplikasi?",
    "Dokumen apa sajakah persyaratan mengajukan JHT bagi PMI yang berpindah menjadi Warga Negara Asing?",
    "Promo apa saja yang tersedia pada aplikasi Jamsostek Mobile (JMO)?",
    "Apa itu BPJS Ketenagakerjaan?",
    "Apa saja program BPJS Ketenagakerjaan?",
    "Siapakah yang dimaksud peserta BPJS Ketenagakerjaan?",
    "Apakah yang dimaksud dengan PHK?",
    "Apakah pegawai BHL (Buruh Harian Lepas) dapat mengikuti program JKP",
    "Bagaimana caranya badan usaha/pemberi kerja dapat menggunakan layanan Payment Reminder System(PRS)?",
    "Peserta BPU sudah melakukan pembayaran iuran namun transaksi pembayaran iuran belum merubah masa perlindungan/saldo JHT belum bertambah, apakah yang harus dilakukan?",
    "Apakah aplikasi Electronic Payment System (EPS)?",
    "Apa perbedaan program Jaminan Hari Tua (JHT) dan Jaminan Pensiun (JP)?",
    "Apa saja pelayanan BPJS Kesehatan yang dapat dimanfaatkan?",
    "Bagaimana Mendaftarkan Bayi Baru Lahir?",
    "Apa itu GRC?",
    "Cara Membayar Iuran",
    "Apa saja pelayanan BPJS Kesehatan yang dapat dimanfaatkan?",
    "Kapan pembayaran iuran paling lambat?",
    "Bagaimana komitmen BPJS Kesehatan terkait gratifikasi?"
 
]

for q in questions:
    print(f"\n{'-'*5}")
    print(f"PERTANYAAN: {q}")
    chatbot_response = ask(q)
    print(f"JAWABAN CHATBOT:\n{chatbot_response}")
    print(f"\n{'-'*5}")

    
    # Membandingkan dengan jawaban asli (jika ada pertanyaan yang sama persis)
    actual_answer = answer_from_doc(q)
    if actual_answer != "Tidak ada jawaban yang cocok persis di dalam dataset.":
        print(f"\n{'-'*50}")
        print(f"\nJawaban Sebenarnya (dari dataset): {actual_answer}")
        print(f"\n{'-'*50}")



 UJI COBA 1: Pertanyaan yang Seharusnya Bisa Dijawab karena berasal dari dataset

-----
PERTANYAAN: Bagaimana cara menginstall aplikasi Jamsostek Mobile (JMO)?


c:\Users\MyBook Hype AMD\miniconda3\envs\h8_env\lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


JAWABAN CHATBOT:
Aplikasi Jamsostek Mobile (JMO) dapat diunduh dan diinstall melalui Play Store atau App Store bagi pengguna smartphone.

-----

--------------------------------------------------

Jawaban Sebenarnya (dari dataset): Aplikasi Jamsostek Mobile (JMO) dapat diunduh dan diinstall melalui Playstore/ Appstore bagi pengguna smartphone .

--------------------------------------------------

-----
PERTANYAAN: Bagaimana cara mengajukan klaim JHT melalui JMO jika memiliki lebih dari 1 kartu?


c:\Users\MyBook Hype AMD\miniconda3\envs\h8_env\lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


JAWABAN CHATBOT:
Untuk mengajukan klaim JHT melalui JMO dengan lebih dari 1 kartu BPJS Ketenagakerjaan, Anda perlu melakukan pengkinian data dan mengisi seluruh nomor kartu yang dimiliki agar saldo dapat digabung. Setelah saldo tergabung dalam satu kartu, Anda dapat melanjutkan proses klaim melalui menu Klaim JHT di aplikasi JMO.  Jika Anda tidak mengetahui nomor kartu BPJS Ketenagakerjaan Anda, silakan hubungi Layanan Masyarakat (Contact Center) 175, email ke care@bpjsketenagakerjaan.go.id, atau kunjungi kantor cabang terdekat untuk verifikasi dan informasi nomor kartu Anda.

-----

--------------------------------------------------

Jawaban Sebenarnya (dari dataset): Tidak ada jawaban.

--------------------------------------------------

-----
PERTANYAAN: Promo apa saja yang tersedia pada aplikasi Jamsostek Mobile (JMO)?


...

JAWABAN CHATBOT:
Pelayanan BPJS Kesehatan yang dapat dimanfaatkan meliputi:  (Mohon maaf, detail pelayanan tidak tersedia di FAQ yang diberikan.  FAQ hanya menanyakan "Apa saja pelayanan BPJS Kesehatan yang dapat dimanfaatkan?" tanpa memberikan rinciannya.)

-----

--------------------------------------------------

Jawaban Sebenarnya (dari dataset): Tidak ada jawaban.

--------------------------------------------------

-----
PERTANYAAN: Kapan pembayaran iuran paling lambat?


c:\Users\MyBook Hype AMD\miniconda3\envs\h8_env\lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


JAWABAN CHATBOT:
Pembayaran iuran paling lambat tanggal 10 (sepuluh) setiap bulan.

-----

--------------------------------------------------

Jawaban Sebenarnya (dari dataset): Tidak ada jawaban.

--------------------------------------------------

-----
PERTANYAAN: Bagaimana komitmen BPJS Kesehatan terkait gratifikasi?


c:\Users\MyBook Hype AMD\miniconda3\envs\h8_env\lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


JAWABAN CHATBOT:
Mohon maaf, saya tidak memiliki informasi mengenai hal tersebut. Saya hanya dapat menjawab pertanyaan seputar layanan BPJS Kesehatan dan BPJS Ketenagakerjaan berdasarkan FAQ yang tersedia.  FAQ yang tersedia tidak memuat informasi mengenai komitmen BPJS Kesehatan terkait gratifikasi.

-----

--------------------------------------------------

Jawaban Sebenarnya (dari dataset): Tidak ada jawaban.

--------------------------------------------------


In [10]:
# TEST CASE 2: Pertanyaan di luar konteks (untuk menguji penolakan)
# Pertanyaan ini untuk memastikan chatbot tidak mengarang jawaban.
print("\n\n UJI COBA 2: Pertanyaan di Luar Konteks ")
questions_out_of_context = [
    "Apa itu Allo Bank?",
    "Bagaimana cara mengajukan KPR?",
    "Siapa penemu listrik?",
    "Apa saja manfaat dari olahraga?",
    "Bagaimana cara membuat kue?",
    "Jelaskan apa itu Quantum Computing?",
    "Apa yang dimaksud dengan Machine Learning?",
    "Gimana cara buat foto HD dengan AI?",
    "Apa itu teknologi blockchain?",
    "Apa itu teknologi quantum?",
    "Apa kopi paling enak saat ini?",
    "Sebutkan Sepatu lari paling enak",
    "Kapan Indonesia Merdeka?",
    "Dimana kantor urusan agama terdekat",
    "Jelaskan tentang skema piala dunia?",
    "Siapa pencetak gol terbanyak piala dunia 2022?",
    "Jumlah total trofi Liverpool berapa?",
    "Hitung luas Kota Surabaya",
    "Dimana tempat syuting film laskar Pelangi?",
    "Dimana bom atom amerika saat perang dunia kedua dibuat?",
    "Bagaimana cara membedakan biji kopi?",
    "Jembatan merah ada Dimana?",
    "Siapa presiden Indonesia pertama?",
    "Siapa penemu alat musik gitar?",
    "Hitung luas lapangan bola."
]

for q in questions_out_of_context:
    print(f"\n{'-'*5}")
    print(f"Pertanyaan: {q}")
    chatbot_answer = ask(q)
    print(f"Jawaban Chatbot:\n{chatbot_answer}")

    # Membandingkan dengan jawaban asli (jika ada pertanyaan yang sama persis)
    actual_answer = answer_from_doc(q)
    if actual_answer != "Tidak ada jawaban yang cocok persis di dalam dataset.":
        print(f"\n{'-'*50}")
        print(f"\nJawaban Sebenarnya (dari dataset): {actual_answer}")
        print(f"\n{'-'*50}")
    




 UJI COBA 2: Pertanyaan di Luar Konteks 

-----
Pertanyaan: Apa itu Allo Bank?


c:\Users\MyBook Hype AMD\miniconda3\envs\h8_env\lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Jawaban Chatbot:
Mohon maaf, saya tidak memiliki informasi mengenai Allo Bank. Saya hanya dapat menjawab pertanyaan seputar layanan BPJS Kesehatan dan BPJS Ketenagakerjaan.

--------------------------------------------------

Jawaban Sebenarnya (dari dataset): Tidak ada jawaban.

--------------------------------------------------

-----
Pertanyaan: Bagaimana cara mengajukan KPR?


c:\Users\MyBook Hype AMD\miniconda3\envs\h8_env\lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Jawaban Chatbot:
Mohon maaf, saya tidak memiliki informasi mengenai cara mengajukan KPR (Kredit Pemilikan Rumah) berdasarkan FAQ yang tersedia.  Saya hanya dapat menjawab pertanyaan seputar layanan BPJS Kesehatan dan BPJS Ketenagakerjaan yang tercantum dalam FAQ tersebut.

--------------------------------------------------

Jawaban Sebenarnya (dari dataset): Tidak ada jawaban.

--------------------------------------------------

-----
Pertanyaan: Siapa penemu listrik?


...

Jawaban Chatbot:
Mohon maaf, saya tidak memiliki informasi mengenai hal tersebut. Saya hanya dapat menjawab pertanyaan seputar layanan BPJS Kesehatan dan BPJS Ketenagakerjaan.

--------------------------------------------------

Jawaban Sebenarnya (dari dataset): Tidak ada jawaban.

--------------------------------------------------

-----
Pertanyaan: Siapa penemu alat musik gitar?


c:\Users\MyBook Hype AMD\miniconda3\envs\h8_env\lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Jawaban Chatbot:
Mohon maaf, saya tidak memiliki informasi mengenai hal tersebut. Saya hanya dapat menjawab pertanyaan seputar layanan BPJS Kesehatan dan BPJS Ketenagakerjaan.

--------------------------------------------------

Jawaban Sebenarnya (dari dataset): Tidak ada jawaban.

--------------------------------------------------

-----
Pertanyaan: Hitung luas lapangan bola.


c:\Users\MyBook Hype AMD\miniconda3\envs\h8_env\lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Jawaban Chatbot:
Mohon maaf, saya tidak memiliki informasi mengenai hal tersebut. Saya hanya dapat menjawab pertanyaan seputar layanan BPJS Kesehatan dan BPJS Ketenagakerjaan.

--------------------------------------------------

Jawaban Sebenarnya (dari dataset): Tidak ada jawaban.

--------------------------------------------------


In [11]:
# TEST CASE 3: Pertanyaan dalam konteks namun ada kesalahan penulisan
print("\n\n UJI COBA 3: Pertanyaan Masih dalam Konteks namun terdapat kesalahan penulisan ")
questions_context_typo = [
    "Aph itu BPJS Ketenagakerajaan?",
    "Apa sajak program BPJS Ketenagakerjaan?",
    "Siapakah yang dimaksud peserta BPJS Ketenagakerajaan?",
    "Apakh yang dimaksud dengan PHK?",
    "Apakah pegawai BHL (Burung Harian Lepas) dapat mengikuti program JKP?",
    "Peserta BPU sudah melakukan pembayaran iuran namun transaksi pembayaran iuran belum merubah masa perlindungan/saldo JHT belum bertambah, apakah yang harus dilakukan?", 
    "Gimana jika lupa PIN EPS?",
    "Apakah itu Perumahan Pekerja?",
    "Dokumen apa sajakah persyaratan mengajukan JHT bagi PMI yang berpindah menjadi Warga Negara Asing?",
    "Kapan BSU tahun 2025 dibagikan?",
    "Dimana obat batuk pertama kali ditemukan",
    "Jika flu sebaiknya minum obat herbal apa",
    "Sopo peserta BPJS ketenagakerjaan",
    "Sejak tanggal berapa pembayaran iuran paling lambat ?",
    "Jika anggota baru di JMO, apakah harus mendaftar di MotionIme?",
    "Apakah itu ShoopePay?",
    "Apa perbedaan program Jaminan Dari Tua (JHT) dan Jaminan Pensiun (JP)?",
    "Gimana pembayaran iuran?",
    "Tanggal pembayaran iuran paling lambat ?",
    "Promo apa sajak yang tersedia pada aplikasi Jamsostek Mobile (JMO)?"
    
]

for q in questions_context_typo:
    print(f"\n{'-'*5}")
    print(f"Pertanyaan: {q}")
    chatbot_answer = ask(q)
    print(f"Jawaban Chatbot:\n{chatbot_answer}")

    
    # Membandingkan dengan jawaban asli (jika ada pertanyaan yang sama persis)
    actual_answer = answer_from_doc(q)
    if actual_answer != "Tidak ada jawaban yang cocok persis di dalam dataset.":
        print(f"\n{'-'*50}")
        print(f"\nJawaban Sebenarnya (dari dataset): {actual_answer}")
        print(f"\n{'-'*50}")
    
    




 UJI COBA 3: Pertanyaan Masih dalam Konteks namun terdapat kesalahan penulisan 

-----
Pertanyaan: Aph itu BPJS Ketenagakerajaan?


c:\Users\MyBook Hype AMD\miniconda3\envs\h8_env\lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Jawaban Chatbot:
Mohon maaf, saya tidak memiliki informasi mengenai hal tersebut. Saya hanya dapat menjawab pertanyaan seputar layanan BPJS Kesehatan dan BPJS Ketenagakerjaan berdasarkan FAQ yang tersedia.  FAQ yang tersedia saat ini tidak menjelaskan secara detail "Apa itu BPJS Ketenagakerjaan?".

--------------------------------------------------

Jawaban Sebenarnya (dari dataset): Tidak ada jawaban.

--------------------------------------------------

-----
Pertanyaan: Apa sajak program BPJS Ketenagakerjaan?


c:\Users\MyBook Hype AMD\miniconda3\envs\h8_env\lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Jawaban Chatbot:
Mohon maaf, saya tidak memiliki informasi mengenai program-program BPJS Ketenagakerjaan secara rinci.  FAQ yang tersedia hanya menjelaskan cara pendaftaran, penahapan kepesertaan, siapa peserta, fungsi BPJS Ketenagakerjaan, dan cara badan usaha mendaftar.  Untuk informasi detail mengenai program-program BPJS Ketenagakerjaan, saya sarankan untuk mengunjungi website resmi BPJS Ketenagakerjaan atau menghubungi kantor cabang terdekat.

--------------------------------------------------

Jawaban Sebenarnya (dari dataset): Tidak ada jawaban.

--------------------------------------------------

-----
Pertanyaan: Siapakah yang dimaksud peserta BPJS Ketenagakerajaan?


...

Jawaban Chatbot:
Mohon maaf, pertanyaan Anda kurang spesifik.  Untuk informasi mengenai pembayaran iuran, mohon jelaskan lebih detail jenis iuran yang Anda tanyakan (BPJS Kesehatan atau BPJS Ketenagakerjaan), dan apakah Anda ingin mengetahui cara pembayaran melalui autodebet,  melalui kanal pembayaran lain, atau informasi lainnya terkait pembayaran iuran.  Informasi yang lebih spesifik akan membantu saya memberikan jawaban yang akurat.

--------------------------------------------------

Jawaban Sebenarnya (dari dataset): Tidak ada jawaban.

--------------------------------------------------

-----
Pertanyaan: Tanggal pembayaran iuran paling lambat ?


c:\Users\MyBook Hype AMD\miniconda3\envs\h8_env\lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Jawaban Chatbot:
Pembayaran iuran paling lambat tanggal 10 (sepuluh) setiap bulan.

--------------------------------------------------

Jawaban Sebenarnya (dari dataset): Tidak ada jawaban.

--------------------------------------------------

-----
Pertanyaan: Promo apa sajak yang tersedia pada aplikasi Jamsostek Mobile (JMO)?


c:\Users\MyBook Hype AMD\miniconda3\envs\h8_env\lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Jawaban Chatbot:
Promo yang tersedia di aplikasi Jamsostek Mobile (JMO) merupakan produk dari berbagai merchant yang bekerjasama dengan BPJS Ketenagakerjaan.

--------------------------------------------------

Jawaban Sebenarnya (dari dataset): Tidak ada jawaban.

--------------------------------------------------


# Hasil Test case

- ## Test Case 1

Total pertanyaan sesuai dataset : 25

Pertanyaan terjawab             : 18

Pertanyaan tidak terjawab       : 7


- ## Test Case 2

Total pertanyaan diluar dataset : 25

Pertanyaan terjawab             : 0

Pertanyaan tidak terjawab       : 25

- ## Test Case 3

Total pertanyaan salah tidak baku/typo : 20

Pertanyaan terjawab             : 11

Pertanyaan tidak terjawab       : 9



# Kesimpulan

Berdasarkan rangkaian test terhadap chatbot yang dibangun dengan model **gemini-1.5-flash-latest** dan berbasis pada databse FAQ resmi BPJS Ketenagakerjaan dan Kesehatan, dapat disimpulkan bahwa:

1. **Kemampuan Kesesuaian**, chatbot mampu memberikan jawaban yang relevan dengan yang berada dalam dataset FAQ. Model memiliki pemahaman konteks yang baik sehingga jawaban cukup akurat. Pada test case 1, pertanyaan yang dirasa terlalu general seperti "Apakah itu promo?" tidak dapat dijawab walaupun terdapat dalam FAQ, hal ini dapat terjadi karena keterbatasan pada tahap pengambilan informasi(retrieval). Pertanyaan terlalu singkat dan general sehingga sistem kesulitan membedakan potongan data (chunk) mana yang paling relevan. Namun jika pertanyaan spesifik tentang BPJS Ketenagakerjaan dan Kesehatan, model mampu menjawan dengan cukup akurat.

2. **Kemampuan Menjaga Batasan Konteks**, chatbot mampu menjaga batasan konteks project, hal ini dibuktikan dengan adanya tes yang dilakukan dengan memberi pertanyaan diluar konteks. Pada tes tersebut chatbot berhasil untuk tidak menjawab semua pertanyaan yang berada diluar konteks dari BPJS Ketenagakerjaan dan Kesehatan. Hal ini didapatkan dari keberhasilan dari prompt yang digunakan.

3. **Kemampuan Membaca pertanyaan yang berada dalam konteks atau terdapat typo inputan user**, chatbot dapat menentukan cukup baik batasan mana pertanyaan yang berada dalam konteks BPJS ketenagakerjaan & Kesehatan. Pada pertanyaan yang masih berada dalam konteks atau terdapat beberapa kesalahan penulisan. Chatbot cukup baik dalam menganalisa pertanyaan yang berada dalam konteks karena dalam test case 3 dapat menjawab 6 dari 9 pertanyaan. Sedangkan pada typo yang dapat merubah suatu makna dalam suatu kata hasil tidak terlalu bagus, namun jika typo atau kesalahan input merupakan kata tanya chatbot mampu menjawab dengan tepat.

**Model gemini-1.5-flash-latest**

***Keunggulan***:
- **Kecepatan dalam menjawab dari model ini sangat tinggi**, sehingga interaksi terasa alami dan dapat membantu dengan cepat dari user.

- **Biaya Rendah**, secara signifikan biaya lebih murah jika dibandingkan model yang lebih besar seperti Gemini 1.5 pro. Harga yang lebih murah tersebut menjadi pilihan yang cocok digunakan untuk chatbot dengan volume interaksi tinggi dimana biaya operasional sangat penting.

- **Model Ringan**, karena model ringan dan cepat cocok untuk lalu lintas percakapan yang tinggi dengan kebutuhan skalabilitas besar.

***Kelemahan***:
- **Akurasi lebih rendah jika dibandingkan versi Pro**, karena model dirasa "ringan" maka akurasi dari model tidak terlalu baik jika dibandingkan versi pro.